In [0]:
from pyspark.sql.functions import col, when, lit, current_timestamp

# Step 1: Load Bronze table (processed)
bronze_df = spark.table("inflation.bronze_daily_prices_processed")

# Step 2: Clean & standardize
silver_df = bronze_df.filter((col("price").isNotNull()) & (col("price") > 0))
silver_df = silver_df.withColumn("unit", col("unit").alias("unit"))
silver_df = silver_df.withColumn(
    "category",
    when(col("category").isin("Food", "Vegetable"), col("category"))
    .otherwise(lit("Other"))
)
silver_df = silver_df.withColumn("ingested_at", current_timestamp())

# Step 3: Save as Silver Delta Table
silver_df.write.format("delta").mode("overwrite").saveAsTable("inflation.silver_daily_prices")

# Step 4: Preview Silver Table
display(spark.table("inflation.silver_daily_prices").orderBy("date", "item_id"))
